<a href="https://colab.research.google.com/github/gabrielbribeiroo/OperationalResearch_UFPB/blob/main/Pratica_PLI_Corte_e_Empregados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula prática: Programação Linear Inteira

## Exercício 1
<sup>Exercício 3.7 do livro `Pesquisa Operacional` de `Arenales, Armentano, Morabito e Yanasse`.</sup>

### Descrição do problema
Em cada dia da semana, uma loja requer um número de empregados em tempo integral, de acordo com a tabela abaixo. Cada empregado deve trabalhar cinco dias consecutivos e descansar dois. Cada empregado recebe R$30 por dia.

| | Segunda | Terça | Quarta | Quinta | Sexta | Sabádo | Domingo |
|:---|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| Empregados | 10 | 6 | 8 | 5 | 9 | 4 | 6 |

Determine o número de empregados em tempo integral de forma a minimizar a despesa total com salários.

### Resolução

In [ ]:
# instalação e importação do pacote mip
!pip install mip
from mip import *

# funcões usadas posteriormente:

# resolve o modelo e mostra os valores das variáveis
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")


# salva modelo em arquivo lp, e mostra o conteúdo
def save(model, filename):
  model.write(filename) # salva modelo em arquivo
  with open(filename, "r") as f: # lê e exibe conteúdo do arquivo
    print(f.read())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 38.5 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


**negrito**#### Dados

$D = \{0, 1, 2, 3, 4, 5, 6\}$: conjunto de dias \\
$e_d$: quantidade mínima de empregados no dia $d \in D$ \\
$D^*_d$: conjunto de dias que antecedem o dia $d$ em, no máximo, 5 dias. Isto é, se um funcionário começar a trabalhar em um dia $p \in D^*_d$, então ele também trabalhará no dia $d$. Observe que o próprio $d$ pertence a $D^*_d$.

In [ ]:
D = range(7)
e = [10, 6, 8, 5, 9, 4, 6]

def Dstar(d):
  p = d + 2  # before first day
  for _ in range(5):
    p = (p + 1) % 7
    yield p

#### Variável
$x_d$: quantidade de empregados que começam a trabalhar no dia $d \in D$ \\

#### Modelo

$$\min \sum_{d \in D} x_d$$
s.t.
$$\sum_{p \in D^*_d} x_p \geq e_d; \forall d \in D$$
$$x_d \geq 0; \forall d \in D$$
$$x_d \in \mathbb{Z}; \forall d \in D$$

In [ ]:
model = Model(sense=MINIMIZE, solver_name=CBC)

x = [model.add_var(var_type=INTEGER, name=f"x_{d}", lb=0) for d in D]

model.objective = xsum(x[d] for d in D)

for d in D:
  model += xsum(x[p] for p in Dstar(d)) >= e[d]

save(model, "model.lp")

\Problem name: 

Minimize
OBJROW: x1 + x2 + x3 + x4 + x5 + x6 + x7
Subject To
constr(0):  x1 + x4 + x5 + x6 + x7 >= 10
constr(1):  x1 + x2 + x5 + x6 + x7 >= 6
constr(2):  x1 + x2 + x3 + x6 + x7 >= 8
constr(3):  x1 + x2 + x3 + x4 + x7 >= 5
constr(4):  x1 + x2 + x3 + x4 + x5 >= 9
constr(5):  x2 + x3 + x4 + x5 + x6 >= 4
constr(6):  x3 + x4 + x5 + x6 + x7 >= 6
Bounds
Integers
x1 x2 x3 x4 x5 x6 x7 
End



In [ ]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 11.00

Solution:
x1 = 5.00
x2 = 0.00
x3 = 1.00
x4 = 3.00
x5 = 0.00
x6 = 2.00
x7 = 0.00


## Exercício 2
<sup>Exercício da lista do Professor Marcone Jamilson (UFOP)</sup>

### Descrição do problema
Uma serralheria dispõe de barras de 6 metros de comprimento que devem ser cortadas para obter barras menores nos seguintes tamanhos: 50 barras de 2 metros, 60 barras de 3 metros e 90 barras de 4 metros. Elabore um modelo de programação linear inteira que minimize a quantidade de barras utilizadas.

Dica: enumere as possíveis formas de se cortar uma barra de 6 metros em barras menores dos tamanhos listados acima.

### Resolução

Neste exercício, precisaremos construir uma tabela, a partir das informações disponíveis no enunciado, que indique cada uma das maneiras de cortar a barra de 6 metros e quantas barras menores cada um desses cortes fornecerá:

| | Barra 2m | Barra 3m | Barra 4m | Resto |
|:---|:---:|:---:|:---:|:---:|
| Corte 1 | 3 | 0 | 0 | 0 |
| Corte 2 | 1 | 1 | 0 | 1 |
| Corte 3 | 1 | 0 | 1 | 0 |
| Corte 4 | 0 | 2 | 0 | 0 |

#### Modelo

$x_i$: quantidade do corte $i$ realizado. \\

$$\min x_1 + x_2 + x_3 + x_4$$
s.t.

$$3 x_1 + x_2 + x_3 \geq 50$$
$$x_2 + 2 x_4 \geq 60$$
$$x_3 \geq 90$$
$$x \geq 0$$
$$x \in \mathbb{Z}$$

No modelo acima, $x$ é o vetor de variáveis, ou seja, $x=(x_1,x_2,x_3,x_4)$. Logo, as duas últimas restrições do modelo determinam que as quatro variáveis devem assumir valores inteiros não negativos.

In [ ]:
model = Model(sense=MINIMIZE, solver_name=CBC)

x = [None] + [model.add_var(var_type=INTEGER, name=f"x_{i}", lb=0) for i in range(1, 5)]

model.objective = x[1] + x[2] + x[3] + x[4]

model += 3*x[1] + x[2] + x[3] >= 50
model += x[2] + 2*x[4] >= 60
model += x[3] >= 90

save(model, "model2.lp")

\Problem name: 

Minimize
OBJROW: x1 + x2 + x3 + x4
Subject To
constr(0):  3 x1 + x2 + x4 >= 50
constr(1):  x2 >= 90
constr(2):  2 x3 + x4 >= 60
Bounds
Integers
x1 x2 x3 x4 
End



In [ ]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 120.00

Solution:
x1 = 0.00
x2 = 90.00
x3 = 30.00
x4 = 0.00
